# NUMBER PLATE RECOGNITION USING ARTIFICIAL INTELLENGENCE

By **Kasasira Charles Derrick**<br/>
21/U/05662/EVE<br />
Student Makerere University


### Objective:
The objective of this notebook to train the model for the Number Plate Reconition System

In [3]:
#importing the dataset from google drive.
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing Libraries

In [6]:
pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 29.9 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [35]:
pip install Image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 KB 10.0 MB/s eta 0:00:00
  Created wheel for Image: filename=image-1.5.33-py2.py3-none-any.whl size=19494 sha256=ba07ce2343b7c97ba166dd18927de3ef34fcd20b8a741dd5d5009eabc553876e
  Stored in directory: /root/.cache/pip/wheels/ac/30/5c/a8b33888bea3507eda7c924a143d34b2390d2ca5b145b327b5
Successfully built Image


In [7]:
import cv2
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import pytesseract as pt
from PIL import Image
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

## Preprocessing the Dataset

In [8]:
path = glob('/content/drive/MyDrive/Number_Plate_Dateset/images/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [9]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/content/drive/MyDrive/Number_Plate_Dateset/im...,233,432,88,151
1,/content/drive/MyDrive/Number_Plate_Dateset/im...,198,349,107,143
2,/content/drive/MyDrive/Number_Plate_Dateset/im...,265,725,247,347
3,/content/drive/MyDrive/Number_Plate_Dateset/im...,195,383,320,358
4,/content/drive/MyDrive/Number_Plate_Dateset/im...,58,221,198,271


In [10]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/content/drive/MyDrive/Number_Plate_Dateset/images',filename_image)
    return filepath_image
getFilename(filename)

'/content/drive/MyDrive/Number_Plate_Dateset/images/N145.jpeg'

In [12]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]

['/content/drive/MyDrive/Number_Plate_Dateset/images/N145.jpeg',
 '/content/drive/MyDrive/Number_Plate_Dateset/images/N131.jpeg',
 '/content/drive/MyDrive/Number_Plate_Dateset/images/N152.jpeg',
 '/content/drive/MyDrive/Number_Plate_Dateset/images/N176.jpeg',
 '/content/drive/MyDrive/Number_Plate_Dateset/images/N128.jpeg',
 '/content/drive/MyDrive/Number_Plate_Dateset/images/N160.jpeg',
 '/content/drive/MyDrive/Number_Plate_Dateset/images/N104.jpeg',
 '/content/drive/MyDrive/Number_Plate_Dateset/images/N156.jpeg',
 '/content/drive/MyDrive/Number_Plate_Dateset/images/N169.jpeg',
 '/content/drive/MyDrive/Number_Plate_Dateset/images/N148.jpeg']

Data

In [13]:
file_path = image_path[1] #path of our image N2.jpeg
# img = cv2.imread(file_path) #read the image
# xmin-1804/ymin-1734/xmax-2493/ymax-1882 
# img = io.imread(file_path) #Read the image
img = Image.open('/content/drive/MyDrive/Number_Plate_Dateset/images/N1.jpeg') #read the image using Pillow
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 8 - N2.jpeg with bounding box')
fig.add_shape(type='rect',x0=1804, x1=2493, y0=1734, y1=1882, xref='x', yref='y',line_color='cyan')

UnidentifiedImageError: ignored

In [14]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)


UnidentifiedImageError: ignored

In [15]:


# Convert data to array
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)



In [16]:
# Split the data into training and testing set using sklearn.
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

ValueError: ignored

In [17]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)


# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

219055592/219055592 [==============================] - 1s 0us/step


In [18]:
# Complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [20]:
model.save('./object_detection.h5')

Predication

In [21]:
# Load model
model = tf.keras.models.load_model('./object_detection.h5')
print('Model loaded Sucessfully')

Model loaded Sucessfully


In [23]:
path = '/content/drive/MyDrive/Number_Plate_Dateset/TEST/TEST.jpeg'
image = load_img(path) # PIL object
image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
image1 = load_img(path,target_size=(224,224))
image_arr_224 = img_to_array(image1)/255.0  # Convert into array and get the normalized output

# Size of the orginal image
h,w,d = image.shape
print('Height of the image =',h)
print('Width of the image =',w)

UnidentifiedImageError: ignored

In [24]:
dtrain = os.listdir('/content/drive/MyDrive/Number_Plate_Dateset/images')
dtest = os.listdir('/content/drive/MyDrive/Number_Plate_Dateset/images')


In [31]:
TRAIN_PATH = '/content/drive/MyDrive/Number_Plate_Dateset/images'
TEST_PATH = '/content/drive/MyDrive/Number_Plate_Dateset/image/test'

In [25]:


len(dtrain)



453

In [26]:
img_path = []
txt_path = []
for x in range(len(dtrain)):
    if dtrain[x].endswith('.jpeg'):
        suf = dtrain[x][:-5]
        img_path.append(dtrain[x])
        txt_path.append(suf + '.txt')

In [27]:
img_test_path = []
txt_test_path = []
for x in range(len(dtest)):
    if dtest[x].endswith('.jpeg'):
        suf = dtest[x][:-5]
        img_test_path.append(dtest[x])
        txt_test_path.append(suf + '.txt')

In [32]:
data = []
output = []
for i in range(len(img_path)):
    image = os.path.join(TRAIN_PATH, img_path[i])
    img_arr = cv2.imread(image)
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    lab = img_path[i][:-5]
    with open(os.path.join(TRAIN_PATH, lab+'.txt'), 'r') as file:
        doc = file.read()
        file.close()
    cases = doc.split('\n')
    box = []
    for case in cases:
        nxmin,nxmax,nymin,nymax = case.split(' ')[1:]
        label_norm = (float(nxmin),float(nxmax),float(nymin),float(nymax))
        box.append(label_norm)
    data.append(norm_load_image_arr)
    output.append(box)

UnidentifiedImageError: ignored

In [33]:
data_test = []
output_test = []
for i in range(len(img_test_path)):
    image = os.path.join(TEST_PATH, img_test_path[i])
    img_arr = cv2.imread(image)
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    lab = img_test_path[i][:-5]
    with open(os.path.join(TEST_PATH, lab+'.txt'), 'r') as file:
        doc = file.read()
        file.close()
    cases = doc.split('\n') 
    box = []
    for case in cases:
        nxmin,nxmax,nymin,nymax = case.split(' ')[1:]
        label_norm = (float(nxmin),float(nxmax),float(nymin),float(nymax))
        box.append(label_norm)
    data_test.append(norm_load_image_arr)
    output_test.append(box)

FileNotFoundError: ignored

In [34]:
x_train,x_test,y_train,y_test = np.array(data,dtype=np.float32), np.array(data_test,dtype=np.float32), np.array(output,dtype=np.float32), np.array(output_test,dtype=np.float32)

In [35]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)


# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [36]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

In [40]:
history = model.fit(x=x_train,y=y_train,batch_size=20,epochs=20,
                    validation_data=(x_test,y_test))

Epoch 1/20


ValueError: ignored

In [41]:


history = model.fit(x=x_train,y=y_train,batch_size=20,epochs=20,
                    validation_data=(x_test,y_test))



Epoch 1/20


ValueError: ignored

In [42]:
# print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('accuracy.png')
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('loss.png')

NameError: ignored